In [1]:
import torch
import torchvision

In [2]:
def calc_acc(preds,labels):
  _,pred_max=torch.max(preds,1)
  acc=torch.sum(pred_max==labels.data,dtype=torch.float64)/len(preds)
  return acc

In [12]:
class MyModel(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1=torch.nn.Linear(784,128)
    self.fc2=torch.nn.Linear(128,32)
    self.fc3=torch.nn.Linear(32,16)
    self.fc4=torch.nn.Linear(16,10)
    # self.fc5=torch.nn.Linear(64,32)
    # self.fc6=torch.nn.Linear(32,16)
    # self.fc7=torch.nn.Linear(16,10)
  def forward(self,x):
    x=x.reshape((x.shape[0],784))
    x=self.fc1(x)
    x=torch.relu(x)
    x=self.fc2(x)
    x=torch.relu(x)
    x=self.fc3(x)
    x=torch.relu(x)
    x=self.fc4(x)
    x=torch.relu(x)
    # x=self.fc5(x)
    # x=torch.relu(x)
    # x=self.fc6(x)
    # x=torch.relu(x)
    # x=self.fc7(x)
    # x=torch.relu(x)
    return x


In [13]:
device=torch.device("cuda")
model=MyModel()
model=model.to(device)
model.train(True)

MyModel(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=10, bias=True)
)

In [14]:
#hyper parameter
batch_size=64
epochs=10
lr=0.01

In [15]:
transform=torchvision.transforms.Compose([
                                torchvision.transforms.ToTensor(),
                                torchvision.transforms.Normalize((0),(1)),
])

dataset=torchvision.datasets.FashionMNIST("./dataset",train=True,transform=transform,download=True)
train_data_loader=torch.utils.data.DataLoader(dataset,batch_size=batch_size,shuffle=True)

In [16]:
optimizer=torch.optim.Adam(model.parameters(),lr=lr)
loss_function=torch.nn.CrossEntropyLoss()


In [17]:
for epoch in range(epochs):
  train_loss=0.0
  train_acc=0.0
  for images,labels in train_data_loader:
    images=images.to(device)
    labels=labels.to(device)

    optimizer.zero_grad()
    pred=model(images)
    loss=loss_function(pred,labels)
    loss.backward()
    optimizer.step()
    train_loss+=loss
    train_acc+=calc_acc(pred,labels)
  
  total_loss=train_loss/len(train_data_loader)
  total_acc=train_acc/len(train_data_loader)
  print(f"epochs: {epoch} , loss: {total_loss}, acc: {total_acc}")

epochs: 0 , loss: 1.0861618518829346, acc: 0.634878065031983
epochs: 1 , loss: 0.8871561288833618, acc: 0.6902985074626865
epochs: 2 , loss: 0.8304057717323303, acc: 0.6997434701492538
epochs: 3 , loss: 0.8103877902030945, acc: 0.7041078091684435
epochs: 4 , loss: 0.7961141467094421, acc: 0.7075892857142857
epochs: 5 , loss: 0.793221652507782, acc: 0.7085054637526652
epochs: 6 , loss: 0.7805495262145996, acc: 0.7123034381663113
epochs: 7 , loss: 0.7799817323684692, acc: 0.7112206823027718
epochs: 8 , loss: 0.7779161334037781, acc: 0.7104710820895522
epochs: 9 , loss: 0.769792377948761, acc: 0.7141191364605544


In [18]:
torch.save(model.state_dict(),"fashion.pth")

In [19]:
#infrence
import cv2
import numpy as np

In [ ]:
model.eval()
img=cv2.imread('test1.png')
img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
img=cv2.resize(img,(28,28))
tensor=transform(img).unsqueeze(0)
tensor=tensor.to(device)
pred=model(tensor)
# pred=pred.argmax()

pred=pred.cpu().detach().numpy()
out=np.argmax(pred)
out